In [ ]:
#Code to count files in real and fake FileExistsError


test_loss, correct_predictions, total_predictions = 0, 0, 0
real_as_real, real_as_fake, fake_as_real, fake_as_fake = 0, 0, 0, 0
processed_images = set()

for i, (X_test, y_test) in enumerate(test_dataloader):
    X_test, y_test = X_test.to(device), y_test.float().to(device)

    for j in range(len(X_test)):
        image_index = i * test_dataloader.batch_size + j
        image_name = test_dataloader.dataset.data[image_index][0]
        true_label = test_dataloader.dataset.data[image_index][1]

        if image_name in processed_images:
            continue  # Skip already processed images

        processed_images.add(image_name)  # Mark the image as processed

        print(f"Image: {image_name}, True Label: {true_label}")

        if true_label == 0:
            real_as_real += 1
        elif true_label == 1:
            fake_as_fake += 1

        print(f"Real Images: {real_as_real}, Fake Images: {fake_as_fake}")

print(f"Real images: {real_as_real}")
print(f"Fake images: {fake_as_fake}")

In [ ]:
test_loss, correct_predictions, total_predictions = 0, 0, 0
real_as_real, real_as_fake, fake_as_real, fake_as_fake = 0, 0, 0, 0

for i, (X_test, y_test) in enumerate(test_dataloader):
    X_test, y_test = X_test.to(device), y_test.float().to(device)

    for j in range(len(X_test)):
        image_index = i * test_dataloader.batch_size + j
        image_name = test_dataloader.dataset.data[image_index][0]
        true_label = test_dataloader.dataset.data[image_index][1]
        #predicted_label = test_pred_class[j].item()
        print(f"Image: {image_name}, True Label: {true_label}")
        #print(f"Image: {image_name}, True Label: {classes[int(true_label)]}")

        if true_label == 0:
            real_as_real += 1
        elif true_label == 1:
            fake_as_fake += 1

        print(f"Real Images: {real_as_real}, Fake Images: {fake_as_fake}")

    print(f"Real images: {real_as_real}")
    print(f"Fake images: {fake_as_fake}")

In [ ]:
def compute_mean_and_std(data_dir):
    """
    Compute per-channel mean and std of the dataset (to be used in transforms.Normalize())
    """

    cache_file = "mean_and_std.pt"
    if os.path.exists(cache_file):
        print(f"Reusing cached mean and std")
        d = torch.load(cache_file)

        return d["mean"], d["std"]

    ds = datasets.ImageFolder(
        data_dir, transform=transforms.Compose([transforms.ToTensor()])
    )
    dl = torch.utils.data.DataLoader(
        ds, batch_size=1, num_workers=multiprocessing.cpu_count()
    )

    mean = 0.0
    for images, _ in tqdm(dl, total=len(ds), desc="Computing mean", ncols=80):
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
    mean = mean / len(dl.dataset)

    var = 0.0
    npix = 0
    for images, _ in tqdm(dl, total=len(ds), desc="Computing std", ncols=80):
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        var += ((images - mean.unsqueeze(1)) ** 2).sum([0, 2])
        npix += images.nelement()

    std = torch.sqrt(var / (npix / 3))

    # Cache results so we don't need to redo the computation
    torch.save({"mean": mean, "std": std}, cache_file)

    return mean, std

Function to load and save models

In [ ]:
from datetime import datetime

# Save the trained model with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_save_path = f"deepfake_detect_model_{timestamp}.pth"
torch.save(model_0.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

# Load the trained model
loaded_model = DeepFakeDetectV0(output_shape=1)
loaded_model.load_state_dict(torch.load(model_save_path))
loaded_model.to(device)
print("Model loaded successfully")